# ID2222 Data Mining, Homework 2
# **Discovery of Frequent Itemsets and Association Rules**

Brando Chiminelli, Tommaso Praturlon

November 21th, 2022

The goal of this notebook is to ...

## Import libraries and read the dataset

In order to run this notebook you need to import the dataset at this address (https://canvas.kth.se/courses/36211/files/5772174/download?wrap=1) in a 'data' directory.

In [11]:
import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt

PATH_TO_DATA = "../data/T10I4D100K.dat"
df_market = pd.read_csv(PATH_TO_DATA, header=None)
print("Data read successfully!")
# Delete duplicates from the dataset in the columns title and text

df_market.head()

Data read successfully!


,0
0,25 52 164 240 274 328 368 448 538 561 630 687 ...
1,39 120 124 205 401 581 704 814 825 834
2,35 249 674 712 733 759 854 950
3,39 422 449 704 825 857 895 937 954 964
4,15 229 262 283 294 352 381 708 738 766 853 883...


## Finding frequent itemsets with support at least s

Remind that an association rule is an implication X → Y, where X and Y are itemsets such that X∩Y=∅. Support of rule X → Y is the number of transactions that contain X⋃Y. Confidence of rule X → Y is the fraction of transactions containing X⋃Y in all transactions that contain X.

TASK

You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions.

The A-Priori algorithm is based on the rule of monotonic increase of the monotonicity of support: if a set I of items is frequent, then so is every subset of I.

The threshold s of the support should be set sufficiently hihg that not so many frequent itemsets are together. As a rule of thumb, s is 1% of the number of baskets.

In [12]:
S_SUPPORT = 0.01 

#print(df_market.iloc[0])
#print(df_market.columns[0])
df_market[df_market.columns[0]]
print(df_market[df_market.columns[0]])
#df_market_ls = df_market.values.tolist()
#print(type(df_market_ls[0][0]))
#type(int(df_market_ls[0]))

#df_market.apply(tuple, axis=1).tolist()
#print(df_market)
type(df_market)

0        25 52 164 240 274 328 368 448 538 561 630 687 ...
1                  39 120 124 205 401 581 704 814 825 834 
2                          35 249 674 712 733 759 854 950 
3                  39 422 449 704 825 857 895 937 954 964 
4        15 229 262 283 294 352 381 708 738 766 853 883...
                               ...                        
99995                          39 132 623 704 795 825 853 
99996      72 120 241 268 351 354 545 567 606 749 841 893 
99997                              82 313 406 421 438 629 
99998       69 75 140 175 285 494 598 614 641 723 829 871 
99999    32 239 372 419 448 510 540 581 674 752 802 844...
Name: 0, Length: 100000, dtype: object


pandas.core.frame.DataFrame

First iteration: find frequent items

In [14]:
from itertools import combinations

# items must have at least a frequence of 2
MIN_SUPPORT = 2
MIN_CONFIDENCE = 50.0

# dictionary containing all frequencies for frequent items
c1 = dict()
# take all the baskets with their items
df_baskets = df_market[df_market.columns[0]]

# for every basket take the item and if it already exists
# in the dictionary count +1
for basket in df_baskets:
    # print(basket)
    basket = basket.split() # split the string of items
    for item in basket:
        item = int(item) # convert an item to int
        c1[item] = c1.get(item,0) + 1 # get gives the i value, if not found, gives 0
# find minimum frequency among items 
# set MIN_SUPPORT to be above minimum frequency

# delete non-frequent items
for item in list(c1): # c1 is a list of dictionaries (1:6, where 1 is the value and 6 the counter)
    if c1[item]<MIN_SUPPORT:
        del c1[item]

items = list(c1.keys()) # list of all different frequent items
support = [c1] # list of dictionaries
print(support)
print("List of frequent items:\n", items)

{25: 1395, 52: 1983, 164: 744, 240: 1399, 274: 2628, 328: 663, 368: 7828, 448: 1370, 538: 3982, 561: 2783, 630: 1523, 687: 1762, 730: 602, 775: 3771, 825: 3085, 834: 1373, 39: 4258, 120: 4973, 124: 294, 205: 3605, 401: 3667, 581: 2943, 704: 1794, 814: 1672, 35: 1984, 249: 615, 674: 2527, 712: 845, 733: 1141, 759: 788, 854: 2847, 950: 1463, 422: 1255, 449: 1890, 857: 1588, 895: 3385, 937: 4681, 954: 366, 964: 1518, 15: 458, 229: 2281, 262: 660, 283: 4082, 294: 1445, 352: 902, 381: 2959, 708: 1090, 738: 2129, 766: 6265, 853: 1804, 883: 4902, 966: 3921, 978: 1141, 26: 527, 104: 1158, 143: 1417, 320: 540, 569: 2835, 620: 2100, 798: 3103, 7: 997, 185: 1529, 214: 1893, 350: 3069, 529: 7057, 658: 1881, 682: 4132, 782: 2767, 809: 2163, 849: 459, 947: 3690, 970: 2086, 979: 132, 227: 1818, 390: 2685, 71: 3507, 192: 2004, 208: 1483, 272: 659, 279: 3014, 280: 2108, 300: 582, 333: 483, 496: 1428, 530: 1263, 597: 2883, 618: 1337, 675: 2976, 720: 3864, 855: 939, 914: 4037, 932: 1786, 183: 3883, 193: 

Second iteration: to find frequent combinations of items among frequent items

In [15]:
# for every possible length of boundles, (a, b), (a, c, d), (e, f, g, w), ...
for i in range(2,len(items)):
    s = dict()
    # for every combinations of i items
    for combo in combinations(items,i):
        # iterate again in every basket of the original dataframe
        for basket in df_baskets:
            if set(combo).issubset(basket):
                s[combo] = s.get(combo,0) + 1
        if s.get(combo) and s[combo]<MIN_SUPPORT:
            del s[combo]
    if not s:
        break
    support.append(s)
print(support)

KeyboardInterrupt: 

In [ ]:
rules = dict()
for combo in support[-1]:
    for item in combo:
        c = list(combo)
        c.remove(item)
        len_c = len(c)
        c = c[0] if len_c == 1 else tuple(c)
        rule_1 = support[-1][combo]/support[0][item]*100
        rule_2 = support[-1][combo]/support[len_c-1][c]*100
        if rule_1>=MIN_CONFIDENCE: rules[f"{item}->{c}"] = rule_1
        if rule_2>=MIN_CONFIDENCE: rules[f"{c}->{item}"] = rule_2

print(rules)